# Teil a - Simple Perzeptron

## Zum aufwärmen -> Optional !

Die Klassifizierung anhand eines Perzeptrons kann formal als binäre oder dichotome Klassifizierung betrachtet werden. Die Ziel-Klassen werden als 1 (positive Klasse) und 0 (negative Klasse) bezeichnet. 



## Aufbau des Perceptrons

<img src="./Figures/perceptron.png" alt="drawing" style="width:800px;"/>


### (1) Input $\vec{x}$


$\vec{x}$ wird als Inputvektor bezeichnet und repräsentiert die Eingangsdaten. Die Werte werden übereinander geschrieben: Spaltenschreibweise.

### (2) Gewicht $\vec{w}$ 

$\vec{w}$ wird als Gewichtungsvektor bezeichnet. Gewichte stellen die Verbindungen zwischen zwei Neuronen her. Bspw. sendet Neuron A ein Signal an Neuron B. Das Gewicht steuert hierbei, wie stark das Signal in Neuron B ankommt. Ein Wert 0..1 verringert das Signal, ein Wert >1 verstärkt das Signal. Das Gewicht regelt hierbei die Signalstärke. 

Lernen in neuronalen Netzen bedeutet die Anpassung der Gewichte.

Zusätzlich zur Spaltenschreibweise, wird der Gewichtsvektor in Zeilenschreibweise dargestellt, d.h. die Werte werden nebeneinander geschrieben. Hierzu wird dem Gewichtsvektor ein hochgestelltes <i>T</i> angehängt, dies bedeutet <i>transponiert</i>. 

Der Grund hierfür ist, dass damit die Multiplikation zwischen den Inputwerten und den Gewichten einfacher beschrieben werden kann: <br>

### (3) Gewichtete Summe $s$

Die gewichtete Summe bildet die Linearkombination bestehend aus dem Inputvektor $\vec{x}$ und dem Gewichtungsvektor $\vec{w}$: <br>
s = ${w}_1 \cdot {x}_1  + {w}_2 \cdot {x}_2 + ... {w}_n \cdot {x}_n $.


Die Summe aller $w_i \cdot x_i$ lässt sich kompakt darstellen als: $s = \sum^{n}_{i=1}w_i \cdot x_i$. 
 

Die Summe der Produkte der Werte von $\vec{x}$ und $\vec{w}$ wird als Skalarprodukt zweier Vektoren abgekürzt. Hierbei werden Spaltenvektoren in Zeilenvektoren transformiert: $s= w^T x$.

Rechenbeispiel:

$
\begin{pmatrix} 1 & 2 & 3 \end{pmatrix}  
\cdot
\begin{pmatrix} 4 \\ 5 \\ 6 \\ 
\end{pmatrix}  
= 1 \cdot 4 + 2\cdot 5 + 3 \cdot 6 = 32
$
.

#### Ermittlung des Skalarprodukts

Die Multiplikation zweier Vektoren kann anhand der Numpy-Funktion <b>dot()</b> umgesetzt werden. Die Multiplikation wird als <i>Skalarprodukt</i>, <i>Dot-Product</i> oder <i>inneres Produkt</i> bezeichnet. Skalarprodukt, weil das Ergebnis der Multiplikation ein Skalar (dh. ein Wert) ist, und kein Vektor. Dem gegenüber führt der Multiplikationsoperator in Python <b>*</b> die Operation Element für Element aus.

Bilden Sie anhand der Vektoren $\vec{v}_1= [1,2,3]$ und $\vec{v}_2 = [4,5,6]$, abgebildet als Numpy-Arrays, das Skalarprodukt und die elementweise Multiplikation und zeigen Sie die Unterschiede auf.

In [1]:
import numpy as np

In [4]:
v1 = np.array([1,2,3])
v2 = np.array([4,5,6])

print("Skalarprodukt", np.dot(v1,v2))
print("Elementweise Multiplikation", v1*v2)

Skalarprodukt 32
Elementweise Multiplikation [ 4 10 18]


### (4) Stufenfunktion $step$(s)

Für die Stufenfunktion wird folgende Funktion definiert: <br>
$
    step(s) = f(x) = \left\{\begin{array}{lr}
        0, & \text{falls } s < \theta \\
        1, & \text{falls } s \geq \theta
        \end{array}\right\} 
$

Unabhängig vom Eingabewert der Stufenfunktion ist das Ergebnis stets entweder 0 oder 1. Jedoch ist sie vom Schwellenwert Theta $\theta$ abhängig, wann der Sprung von 0 auf 1 stattfindet.

### (5) Output $f_{akt}(\vec{x})$

Ausgangslage folgender Überlegung ist die gewichtete Summe mit der Stufenfunktion (Heaviside-Funktion): <br>
${w}_1 \cdot {x}_1  + {w}_2 \cdot {x}_2 + ... {w}_n \cdot {x}_n \geq \theta$.

Durch Umformung der Ungleichung wird $\theta$ auf die linke Seite gebracht. Das Ergebnis ist ein erweiterter Gewichtsvektor, der um eine Dimension für den Schwellenwert erweitert wird. Die neue Dimension wird am Index 0 eingefügt, sodass der ursprüngliche Vektor inklusive Indizes erhalten bleibt: <br>
${w}_1 \cdot {x}_1  + {w}_2 \cdot {x}_2 + ... {w}_n \cdot {x}_n -\theta \geq  0$.

Damit der Schwellenwert an der Stelle 0 eingefügt werden kann, wird $-\theta$ in $w_0$ umbenannt. Der Vektor $\vec{x}$ wird ebenfalls um einen Wert erweitert. Der Wert für den Input $x_0$ wird mit dem Wert 1 besetzt. Somit existiert eine einheitliche Form um bei der Notation $\vec{w}^T \cdot \vec{x}$ zu bleiben. Es gilt: <br>
${w}_1 \cdot {x}_1  + {w}_2 \cdot {x}_2 + ... {w}_n \cdot {x}_n +w_0 \cdot x_0 \geq  0$.

Diese Form der Nullgewichtung wird in der Literatur als <i>Bias-Neuron</i> bezeichnet. Dadurch ergibt sich folgendes Schaubild:

![title](./Figures/perceptron_erweitert.png)

Die Implementierung auf Code-Ebene weicht davon jedoch etwas ab. <br>
Anstatt den Eingangsvektor $\vec{x}$ mit dem Wert 1 zu erweitern, wird der Wert $w_0$ auf die gewichtete Summe addiert. Formal wiefolgt beschrieben (siehe Skript Prof. Link): 

$s = \vec{w}^T \cdot \vec{x} + w_0$.

## Erste Implementierung

Im folgenden werden die bisher gewonnenen Erkenntnisse in Code umgesetzt. Die Datengrundlage liefert folgende Tabelle (OR-Problem).

<img src="./Figures/or-problem.png" alt="drawing" style="width:600px;"/>

Ingesamt handelt es sich hierbei um vier Inputvektoren. 

Für die Fehlerberechnung der Einzelfehler wird für jeden Input-Vektor der berechnete Output mit dem gewünschten Output verglichen. Das Perceptron kann aufgrund der Heaviside-Funktion entsprechend nur 0 oder 1 ausgeben. Der gewünschte Output ist per Definition ebenfalls 0 oder 1. Somit kann die Differenz nur -1, 0, 1 betragen. In diesem Fall gilt es die Betragsfunktion für den Einzelfehler zu berechnen. Denn sonst würde ein Fehler von -1 den Gesamtfehler verringern. <br>

Die Einzelfehler der Input-Vektren werden summiert und somit der Gesamtfehler bestimmt. Der Gesamtfehler stellt die Ermittlungsgenauigkeit des Perzeptrons dar. <br>

Die Gewichtungen sind mit Werten zu belegen. Experimentieren sie mit den Gewichtungen, sehen sie sich die Fehler an und versuchen sie diese anhand der besprochenen Berechnungen nachzuvollziehen. Wählen Sie die Gewichte so, sodass das OR-Problem gelöst werden kann. 

## Implementierung

Die Implementierung erfolgt innerhalb der Klasse <b>SimplePerceptron</b>. Im folgenden werden die einzelnen Methoden und deren Funktionsweise kurz vorgestellt. <br>

### Konstruktor
Hier ist nichts zu implementieren. <br>

### gewichtete_summe()-Methode:
In dieser Methode soll die beschriebene gewichtete Summe $\vec{w}^T \cdot \vec{x} + w_0$ berechnet werden.

### heaviside()-Methode:
Heaviside-Funktion als Stufenfunktion, dh Schwellenwert ist 0.

### perceptron_eval()-Methode:

<b>Gewichtungen</b>: <br>
Die Gewichtungen in <b>self.w</b> werden mit einem Vektor $\mathbb R^{m+1}$ initialisiert, m gibt die Anzahl der Dimensionen (Merkmale) in der Datensammlung an. Dem ersten Element dieses Vektors (dies entspricht der Bias-Einheit) wird ein Wert zugeordnet. Gehen Sie von zwei Merkmalen aus (wie oben beschrieben).<br>

Implementieren Sie den besprochenen Perceptron-Algorithmus mit den folgenden Schritten:
* Berechnung gewichtete Summe
* Anwendung der Heaviside-Funktion
* Ermittlung des Fehlers
* Ermittlung des Gesamtfehlers

Geben Sie die den Gesamtfehler als Rückgabewert der Methode zurück.

In [37]:
# Ueberlegungen zu den Gewichtungen:

# w1 * 0 + w2 * 0 + w0 = 0  -->  w0 < 0
# w1 * 0 + w2 * 1 + w0 = 1  -->  w2 + w0 >= 0
# w1 * 1 + w2 * 0 + w0 = 1  -->  w1 + w0 >= 0
# w1 * 1 + w2 * 1 + w0 = 1  -->  w1 + w2 + w0 >= 0

# Gewaehlte Gewichte: -1.0, 1.1, 1.1
# 1.1 * 0 + 1.1 * 0 - 1.0 = -1.0  -->  w0 < 0
# 1.1 * 0 + 1.1 * 1 - 1.0 = 0.1  -->  w2 + w0 >= 0
# 1.1 * 1 + 1.1 * 0 - 1.0 = 0.1  -->  w1 + w0 >= 0
# 1.1 * 1 + 1.1 * 1 - 1.0 = 1.2  -->  w1 + w2 + w0 >= 0

In [40]:
class SimplePerceptron(object):
    def __init__(self):
        pass
    
    def heaviside(self, summe):
        if summe >= 0:
            return 1
        else:
            return 0
        return None

    def gewichtete_summe(self, x):
        return self.w[1]*x[0] + self.w[2] * x[1] + self.w[0]
        pass
    
    def perceptron_eval(self, X,y):
        self.w = np.array([-1.0, 1.1, 1.1])
        total_error = 0
        for i in range (0, len(X)-1):
            sum = self.gewichtete_summe(X[i])
            result = self.heaviside(sum)
            error = abs(result -y[i])
            total_error += error
        return total_error

### Algorithmus ausführuen und  Gesamtfehler anzeigen

Führen Sie den SimplePerceptron-Algorithmus mit den beschriebenen Daten uns und geben Sie den Gesamtfehler aus.

In [41]:
# 2 dimensionaler Input: x1, x2
# 4 Inputvektoren
X = np.array([[0,0], [0,1], [1,0], [1,1]])

# Die 4 gewünschten Ergebniswerte
y = np.array([0, 1, 1, 1])

# TODO: implement: USE SimplePerceptron
sp = SimplePerceptron()
print(sp.perceptron_eval(X, y))

0
